In [25]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

The **Combined_Jobs_Final.csv** file: has the main jobs data(**title, description, company, etc.**)

The **Job_Views.csv** file: the file with the jobs seeing for the user.

The **Experience.csv**: the file containing the experience from the user.

The **Positions_Of_Interest.csv**: contains the interest the user previously has manifested.

---
## Job Views data

In [26]:
jViews = pd.read_csv('Job_Views.csv')
print(jViews.shape)
jViews.head()

(12370, 14)


,Applicant.ID,Job.ID,Title,Position,Company,City,State.Name,State.Code,Industry,View.Start,View.End,View.Duration,Created.At,Updated.At
0,10000,73666,Cashiers & Valets Needed! @ WallyPark,Cashiers & Valets Needed!,WallyPark,Newark,New Jersey,NJ,NaN,2014-12-12 20:12:35 UTC,2014-12-12 20:31:24 UTC,1129.0,2014-12-12 20:12:35 UTC,2014-12-12 20:12:35 UTC
1,10000,96655,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's,Garden City,New York,NY,NaN,2014-12-12 20:08:50 UTC,2014-12-12 20:10:15 UTC,84.0,2014-12-12 20:08:50 UTC,2014-12-12 20:08:50 UTC
2,10001,84141,Part Time Showroom Sales / Cashier @ Grizzly I...,Part Time Showroom Sales / Cashier,Grizzly Industrial Inc.,Bellingham,Washington,WA,NaN,2014-12-12 20:12:32 UTC,2014-12-12 20:17:18 UTC,286.0,2014-12-12 20:12:32 UTC,2014-12-12 20:12:32 UTC
3,10002,77989,Event Specialist Part Time @ Advantage Sales &...,Event Specialist Part Time,Advantage Sales & Marketing,Simpsonville,South Carolina,SC,NaN,2014-12-12 20:39:23 UTC,2014-12-12 20:42:13 UTC,170.0,2014-12-12 20:39:23 UTC,2014-12-12 20:39:23 UTC
4,10002,69568,Bonefish - Kitchen Staff @ Bonefish Grill,Bonefish - Kitchen Staff,Bonefish Grill,Greenville,South Carolina,SC,NaN,2014-12-12 20:43:25 UTC,2014-12-12 20:43:58 UTC,33.0,2014-12-12 20:43:25 UTC,2014-12-12 20:43:25 UTC


In [27]:
jViews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12370 entries, 0 to 12369
Data columns (total 14 columns):
Applicant.ID     12370 non-null int64
Job.ID           12370 non-null int64
Title            12370 non-null object
Position         12370 non-null object
Company          11790 non-null object
City             12370 non-null object
State.Name       12348 non-null object
State.Code       12348 non-null object
Industry         2199 non-null object
View.Start       12370 non-null object
View.End         10575 non-null object
View.Duration    10575 non-null float64
Created.At       12370 non-null object
Updated.At       12370 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 1.3+ MB


In [28]:
list(jViews)

['Applicant.ID',
 'Job.ID',
 'Title',
 'Position',
 'Company',
 'City',
 'State.Name',
 'State.Code',
 'Industry',
 'View.Start',
 'View.End',
 'View.Duration',
 'Created.At',
 'Updated.At']

In [29]:
jViews = jViews.drop(['Job.ID', 'Title', 'State.Name', 'State.Code', 'Industry', 'View.Start', 'View.End', 'View.Duration', 'Created.At', 'Updated.At'], axis=1)
jViews.head()

,Applicant.ID,Position,Company,City
0,10000,Cashiers & Valets Needed!,WallyPark,Newark
1,10000,Macy's Seasonal Retail Fragrance Cashier - Ga...,Macy's,Garden City
2,10001,Part Time Showroom Sales / Cashier,Grizzly Industrial Inc.,Bellingham
3,10002,Event Specialist Part Time,Advantage Sales & Marketing,Simpsonville
4,10002,Bonefish - Kitchen Staff,Bonefish Grill,Greenville


In [30]:
jViews.isna().sum()

Applicant.ID      0
Position          0
Company         580
City              0
dtype: int64

In [31]:
jViews['jViewCorpus'] = jViews['Position'].map(str) +' '+jViews["Company"] +"  "+ jViews["City"]
jViews = jViews.drop(['Position', 'Company', 'City'], axis=1)
jViews.columns = ['ApplicantID', 'jViewCorpus']
jViews.head()

,ApplicantID,jViewCorpus
0,10000,Cashiers & Valets Needed! WallyPark Newark
1,10000,Macy's Seasonal Retail Fragrance Cashier - Ga...
2,10001,Part Time Showroom Sales / Cashier Grizzly Ind...
3,10002,Event Specialist Part Time Advantage Sales & M...
4,10002,Bonefish - Kitchen Staff Bonefish Grill Green...


In [32]:
jViews.to_csv('jView_df.csv')

---
## Experience data

In [33]:
experience = pd.read_csv('experience.csv')
experience.head()

,Applicant.ID,Position.Name,Employer.Name,City,State.Name,State.Code,Start.Date,End.Date,Job.Description,Salary,Can.Contact.Employer,Created.At,Updated.At
0,10001,Account Manager / Sales Administration / Quali...,Barcode Resourcing,Bellingham,Washington,WA,2012-10-15,NaN,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
1,10001,Electronics Technician / Item Master Controller,Ryzex Group,Bellingham,Washington,WA,2001-12-01,2012-04-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
2,10001,Machine Operator,comptec inc,Custer,Washington,WA,1997-01-01,1999-01-01,NaN,NaN,NaN,2014-12-12 20:10:02 UTC,2014-12-12 20:10:02 UTC
3,10003,maintenance technician,Winn residental,washington,District of Columbia,DC,NaN,NaN,"Necessary maintenance for ""Make Ready"" Plumbin...",10.0,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC
4,10003,Electrical Helper,michael and son services,alexandria,Virginia,VA,NaN,NaN,repair and services of electrical construction,NaN,False,2014-12-12 21:27:05 UTC,2014-12-12 21:27:05 UTC


In [34]:
experience.isna().sum()

Applicant.ID               0
Position.Name            950
Employer.Name             33
City                    3731
State.Name              4058
State.Code              4058
Start.Date              2035
End.Date                3747
Job.Description         2961
Salary                  5855
Can.Contact.Employer    5072
Created.At                 0
Updated.At                 0
dtype: int64

In [35]:
list(experience)

['Applicant.ID',
 'Position.Name',
 'Employer.Name',
 'City',
 'State.Name',
 'State.Code',
 'Start.Date',
 'End.Date',
 'Job.Description',
 'Salary',
 'Can.Contact.Employer',
 'Created.At',
 'Updated.At']

In [36]:
experience = experience.drop(['Employer.Name', 'City', 'State.Name', 'State.Code', 'Start.Date', 'End.Date', 'Job.Description', 'Salary', 'Can.Contact.Employer', 'Created.At', 'Updated.At'], axis=1)
experience.columns = ['ApplicantID', 'Position']
experience.Position = experience.Position
experience.head()

,ApplicantID,Position
0,10001,Account Manager / Sales Administration / Quali...
1,10001,Electronics Technician / Item Master Controller
2,10001,Machine Operator
3,10003,maintenance technician
4,10003,Electrical Helper


As we see, there are sometimes more than one application for the applicant. Those will be marged to one.

In [37]:
experience.Position = experience.Position.fillna(' ')
experience = experience.groupby('ApplicantID', sort=False)['Position'].apply(' '.join).reset_index()
experience.head(20)

,ApplicantID,Position
0,10001,Account Manager / Sales Administration / Quali...
1,10003,maintenance technician Electrical Helper tech...
2,10004,shift superviveur
3,10005,All Star Houseman
4,10007,Bartender Bar Manager / Head Bartender Bartender
5,10008,"non-food stocker, photo Registrar Registrar ca..."
6,1001,Sales Associate
7,10011,driver helper Cashier customer assistant
8,10016,Host
9,10020,IT Technician II Information Technology Specia...


In [38]:
experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3790 entries, 0 to 3789
Data columns (total 2 columns):
ApplicantID    3790 non-null int64
Position       3790 non-null object
dtypes: int64(1), object(1)
memory usage: 59.3+ KB


In [39]:
experience.to_csv('experience_df.csv')

---
## Position of Interest

In [40]:
poi = pd.read_csv('Positions_Of_Interest.csv')
poi = poi.sort_values(by='Applicant.ID')
poi.head()

,Applicant.ID,Position.Of.Interest,Created.At,Updated.At
6437,96,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1156,153,Barista,2014-08-14 15:56:43 UTC,2015-02-18 02:35:06 UTC
1155,153,Host,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1154,153,Server,2014-08-14 15:56:42 UTC,2015-02-26 20:35:12 UTC
1158,153,Sales Rep,2014-08-14 15:56:47 UTC,2015-03-02 02:13:08 UTC


In [41]:
poi = poi.drop(['Created.At', 'Updated.At'], axis=1)
poi.columns = ['ApplicantID', 'POI']
# poi.POI = poi.POI.str.lower()
poi.POI = poi.POI.fillna(' ')
poi.head(10)

,ApplicantID,POI
6437,96,Server
1156,153,Barista
1155,153,Host
1154,153,Server
1158,153,Sales Rep
1157,153,Customer Service Rep
1952,256,Host
1957,256,Production Area
1956,256,Sales Rep
1955,256,Customer Service Rep


As we see, there are sometimes more than one position of intrest. Those will be marged to one.

In [42]:
poi = poi.groupby('ApplicantID', sort=True)['POI'].apply(' '.join).reset_index()
poi.head(10)

,ApplicantID,POI
0,96,Server
1,153,Barista Host Server Sales Rep Customer Service...
2,256,Host Production Area Sales Rep Customer Servic...
3,438,Customer Service Rep Barista Host Server
4,568,Receptionist Customer Service Rep Book Keeper
5,601,Customer Service Rep Server Line Cook
6,867,Driver Customer Service Rep Book Keeper Chef
7,938,Host Receptionist Customer Service Rep Server
8,1210,Line Cook Host Training Community Relations Cu...
9,1251,Fine Arts Retail Part-Time Travel


In [43]:
poi.to_csv('poi_df.csv')

---
# Marge DataFrames

In [44]:
user_df = jViews.merge(experience, how='outer', left_on='ApplicantID', right_on='ApplicantID').fillna(' ')
user_df = user_df.sort_values(by='ApplicantID')
user_df.head()

,ApplicantID,jViewCorpus,Position
13012,2,,Writer for the Uloop Blog Volunteer
13487,3,,Prep Cook Server Marketing Intern
14628,6,,Project Assistant
15044,8,,"deli clerk,server, cashier, food prep, order t..."
12464,11,,Cashier


In [45]:
user_df = user_df.merge(poi, how='outer', left_on='ApplicantID', right_on='ApplicantID').fillna('')
user_df = user_df.sort_values(by='ApplicantID')
user_df.head(10)

,ApplicantID,jViewCorpus,Position,POI
0,2,,Writer for the Uloop Blog Volunteer,
1,3,,Prep Cook Server Marketing Intern,
2,6,,Project Assistant,
3,8,,"deli clerk,server, cashier, food prep, order t...",
4,11,,Cashier,
5,12,,Server Rec Leader 1 IT Program Management Intern,
6,13,,Assistant OfficA assistant Cashier,
7,14,,Waitress Host Lifeguard,
8,15,,,
9,18,,Barista/Cashier Server Receptionist,


In [46]:
user_df['Corpus'] = user_df.jViewCorpus.map(str)+' '+user_df.Position+' '+user_df.POI
user_df = user_df.drop(['jViewCorpus', 'Position', 'POI'], axis=1)
user_df.Corpus = user_df.Corpus.fillna(' ')
user_df.columns = ['ApplicantID', 'Corpus']

user_df.head()

,ApplicantID,Corpus
0,2,Writer for the Uloop Blog Volunteer
1,3,Prep Cook Server Marketing Intern
2,6,Project Assistant
3,8,"deli clerk,server, cashier, food prep, order..."
4,11,Cashier


In [47]:
# drop all the rows with a empty corpus. (This is an expensive operation. Would be good to optimize this block)
user_df = user_df.drop(user_df[user_df.Corpus == ' '].index, axis=0)
user_df.Corpus = user_df.Corpus.str.lower()
user_df.Corpus = user_df.Corpus.str.replace('[^a-z \n\.]', ' ').fillna(' ')
user_df = user_df.sort_values(by='ApplicantID')
user_df.head()

,ApplicantID,Corpus
0,2,writer for the uloop blog volunteer
1,3,prep cook server marketing intern
2,6,project assistant
3,8,deli clerk server cashier food prep order...
4,11,cashier


In [48]:
user_df.to_csv('user_df.csv', index=False)